In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from matplotlib import animation
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Material teórico

[Slides](https://docs.google.com/presentation/d/1IJ2n8X4w8pvzNLmpJB-ms6-GDHWthfsJTFuyUqHfXg8/edit?usp=sharing)

# Procesamiento digital de imágenes

Una imagen es una colección de pixeles ordenados

En estándar RGB cada pixel corresponde a 3 valores enteros de 8 bit (256 niveles). Combinándolos formamos colores (aproximadamente 16.7M)

Otra codificación usual para los pixeles consiste en usar un número entre cero y uno para cada canal (color)

El estándar RGBA añade un canal que representa la opacidad

Las imágenes en escala de grises y sin opacidad se pueden representar usando un canal

In [ ]:
img = plt.imread('cameraman.png')
img_bw = 0.2989*img[:, :, 0] + 0.587*img[:, :, 1]+ 0.114*img[:, :, 2]

display(img_bw.shape)
display(img_bw.dtype)

plt.figure(figsize=(4, 4))
plt.imshow(img_bw, cmap=plt.cm.Greys_r);

Para nuestros sistemas digitales una imagen es una arreglo multidimensional y podemos operarlo como tal

A que corresponde este segmento del arreglo?

In [ ]:
subimg = np.copy(img_bw[50:100, 120:180])
display(subimg)

In [ ]:
plt.figure(figsize=(3, 3))
plt.imshow(subimg, cmap=plt.cm.Greys_r);

Y este segmento?

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 3))
ax[1].plot(subimg[30, :])
ax[0].imshow(subimg[30:31, :], cmap=plt.cm.Greys_r);

## Convolución  y correlación cruzada discreta

Una herramienta clásica de procesamiento digital de señales es la **convolución**

La operación de convolución entre dos señales unidimensionales discretas es

$$
(f * g) [n] = \sum_{m=-\infty}^\infty  f[m] g[n-m]
$$

y la operación de correlación cruzada es

$$
(f \star g) [n] = \sum_{m=-\infty}^\infty  f[m] g[m+n]
$$

> Para ambas operaciones el resultado es una nueva señal que también depende de  $n$



Por ejemplo el elemento $0$ de $f\star g$ se calcula como

    f[0] g[0] + f[1] g[1] + f[2] g[2] + ...

Luego el elemento $1$ sería

    f[0] g[1] + f[1] g[2] + f[2] g[3] + ...
    
¿Cómo se ve esta operación graficamente?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(7, 4))
ax2 = ax[0].twinx()
data = subimg[0, :]

def filt(k):
    kernel = np.zeros(shape=(len(data),))
    kernel[k:k+5] = 1
    #kernel[k] = 1.; kernel[k+1] = -1.
    return kernel

true_filt = filt(0)[np.absolute(filt(0)) >0]
display(true_filt)
conv_s = scipy.signal.correlate(data, true_filt, mode='valid')


def update(k): 
    ax[0].cla(); ax[1].cla(); ax2.cla();
    ax[0].plot(data)
    ax2.plot(filt(k), c='r')
    ax[1].plot(conv_s); 
    ax[1].scatter(k, conv_s[k], s=100, c='k')
    
anim = animation.FuncAnimation(fig, update, frames=len(conv_s), interval=200, blit=True)

## Filtrado de imágenes con convoluciones

Se puede extender el concepto de convolución a dos dimensiones
$$
(I_1 * I_2) [n_1, n_2] = \sum_{m_1=-\infty}^\infty \sum_{m_2=-\infty}^\infty I_1[m_1, m_2] I_2[n_1-m_2, n_2 - m_2]
$$

donde $n_1$ es el índice de las filas y $n_2$ es el índice de las columnas

#### La convolución entre dos imágenes es una nueva imagen

La imagen $I_1$ es la entrada

La imagen $I_2$ se denomina filtro o kernel de la convolución

La imagen resultante es la imagen filtrada

#### Filtro pasa-bajo

Suaviza, elimina los detalles

In [ ]:
import scipy.signal
D = 3
filt = np.zeros(shape=(D, D))
filt[1:-1, 1:-1] = 1
display(filt)
img_res = scipy.signal.correlate2d(subimg, filt/np.sum(filt), mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

#### Filtro pasa-alto

Resalta los cambios bruscos, elimina las partes "planas"

In [ ]:
filt = np.array([[1., -1.]]*2)
display(filt)
img_res = scipy.signal.correlate2d(subimg, filt, mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

#### Detector de patillas

Detecta patillas de fotografos mirando al horizonte?

In [ ]:
filt = np.ones(shape=(11, 11))
filt[:9, 2:9] = 0

In [ ]:
img_res = scipy.signal.correlate2d(subimg, filt-np.mean(filt), mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
maxloc = np.unravel_index(np.argmax(img_res), shape=img_res.shape)
ax[0].scatter(maxloc[1]+filt.shape[0]//2, maxloc[0]+filt.shape[1]//2, c='r', s=20)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Reds);

> Podríamos aprender filtros para detectar objetos específicos

> Necesitamos aprender los valores de los "píxeles" del kernel

# [Torchvision](https://pytorch.org/docs/stable/torchvision/index.html)

Es una librería utilitaria de PyTorch que facilita considerablemente el trabajo con imágenes

- Sets de benchmark incluidos
- Modelos clásicos pre-entrenados
- Funciones para importar imágenes en distintos formatos
- Funciones de transformación para hacer aumentación de datos en imágenes



    pip3 install torchvision 

#### Cargando dataset MNIST

In [ ]:
import torchvision

mnist_train_data = torchvision.datasets.MNIST(root='/home/phuijse/datasets/',
                                              train=True, download=True, transform=torchvision.transforms.ToTensor())

mnist_test_data = torchvision.datasets.MNIST(root='/home/phuijse/datasets/',
                                              train=False, download=True, transform=torchvision.transforms.ToTensor())

image, label = mnist_train_data[0]
display(len(mnist_train_data), type(image), type(label))
fig, ax = plt.subplots(1, 10, figsize=(8, 2), tight_layout=True)
for k in range(10):
    image, label = mnist_train_data[k]
    ax[k].imshow(image[0, :, :].numpy(), cmap=plt.cm.Greys_r)
    ax[k].axis('off');
    ax[k].set_title(label)

In [ ]:
from torch.utils.data import Subset, DataLoader
import sklearn.model_selection
sss = sklearn.model_selection.StratifiedShuffleSplit(train_size=0.75).split(mnist_train_data.data, mnist_train_data.targets)
train_idx, valid_idx = next(sss)

# Data loader de entrenamiento
#train_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_dataset = Subset(mnist_train_data, train_idx)
#train_data.transforms = train_transforms
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)

# Data loader de validación
#valid_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
valid_dataset = Subset(mnist_train_data, valid_idx)
#train_data.transforms = train_transforms
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=256)

# Red Neuronal Convolucional en PyTorch

Las redes neuronales convolucionales tienen tres tipos de capas

- Capas convolucionales
    
        torch.nn.Conv2D
        
- Capas de pooling

        torch.nn.MaxPool2D
        
- Capas completamente conectadas

        torch.nn.Linear
        
        
> Las capas convolucionales ordenan sus neuronas de tal forma que el resultado es una convolución sobre la imagen

Los parámetros ajustables más importantes de las capas convolucionales son el 
- número de filtros
- tamaño del kernel 
- paso (stride) del kernel

A diferencia de las capas completamente conectadas no es necesario tener tantas neuronas como entradas

In [ ]:
import torch

class mi_red_convolucional(torch.nn.Module):
    
    def __init__(self):
        super(mi_red_convolucional, self).__init__()
        # Extracción de características
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.mpool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3)
        self.mpool2 = torch.nn.MaxPool2d(kernel_size=2)
        # Clasificación
        self.fc1 = torch.nn.Linear(in_features=8*5*5, out_features=10)
        self.fc2 = torch.nn.Linear(in_features=10, out_features=10)
        self.activation = torch.nn.ReLU()
        
    def forward(self, x):
        z = self.mpool1(self.activation(self.conv1(x)))
        z = self.mpool2(self.activation(self.conv2(z)))
        z = z.reshape(-1, 8*5*5)
        z = self.activation(self.fc1(z))
        return self.fc2(z)

In [ ]:
model = mi_red_convolucional()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
use_gpu = False
if use_gpu:
    nnet = nnet.cuda()

# Hiddenlayer objects to track metrics
import hiddenlayer as hl
history1 = hl.History()
canvas1 = hl.Canvas()

#from torch.utils.tensorboard import SummaryWriter
## tensorboard --logdir=/tmp/tensorboard
#writer = SummaryWriter("/tmp/tensorboard/net1")

nepochs = 5
epoch_loss = np.zeros(nepochs, 2)
epoch_acc = np.zeros(nepochs, 2)
for epoch in range(nepochs): 
    epoch_loss, epoch_acc = 0.0, 0.0
    # Train
    for mbdata, mblabel in train_loader:
        if use_gpu:
            mbdata, mblabel = mbdata.cuda(), mblabel.cuda()
        prediction = model.forward(mbdata)
        optimizer.zero_grad()        
        loss = criterion(prediction, mblabel)  
        epoch_loss[k, 0] += loss.item()
        epoch_acc[k, 0] += (torch.nn.Softmax(dim=1)(prediction).argmax(dim=1) == mblabel).sum().item()        
        loss.backward()
        optimizer.step()
    epoch_loss[k, 0] = epoch_loss[k, 0]/len(train_idx)
    epoch_acc[k, 0] = epoch_acc[k, 0]/len(train_idx)
    # Validation
    #writer.add_scalar('Train/Loss', epoch_loss/len(train_idx), epoch)
    #writer.add_scalar('Train/Acc', epoch_acc/len(train_idx), epoch)
    epoch_loss, epoch_acc = 0.0, 0.0
    for mbdata, mblabel in valid_loader:
        if use_gpu:
            mbdata, mblabel = mbdata.cuda(), mblabel.cuda()
        prediction = model.forward(mbdata)
        loss = criterion(prediction, mblabel)  
        epoch_loss[k, 1] += loss.item()
        epoch_acc[k, 1] += (torch.nn.Softmax(dim=1)(prediction).argmax(dim=1) == mblabel).sum().item()        
    #writer.add_scalar('Valid/Loss', epoch_loss/len(valid_idx), epoch)
    #writer.add_scalar('Valid/Acc', epoch_acc/len(valid_idx), epoch)
    epoch_loss[k, 1] = epoch_loss[k, 0]/len(valid_idx)
    epoch_acc[k, 1] = epoch_acc[k, 0]/len(valid_idx)
    history1.log(epoch, loss=epoch_loss[k, 1], accuracy=epoch_acc[k, 1])
    with canvas1: # So that they render together
        canvas1.draw_plot([history1["loss"]])
        canvas1.draw_plot([history1["accuracy"]])
    #time.sleep(0.1)

if use_gpu:
    nnet = model.cpu()
    
#writer.add_graph(nnet)
#writer.close()

### Visualizando los filtros aprendidos

In [ ]:
fig, ax = plt.subplots(1, 8, figsize=(7, 2))
w = model.conv1.weight.data.numpy()

for i in range(8):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')

### Utilizando la red neuronal para clasificar ejemplos de test

In [ ]:
image, label = mnist_test_data[10]
y = torch.nn.Softmax(dim=1)(model.forward(image.unsqueeze(0)))
display(y)
display(torch.argmax(y))
display(label)

plt.figure(figsize=(3, 3))
plt.imshow(image.numpy()[0, :, :], cmap=plt.cm.Greys_r)

In [ ]:
entropy = []
for i in range(len(mnist_test_data)):
    image_test, label_test = mnist_test_data[i]
    sl = model.forward(image_test.unsqueeze(0))
    y = torch.nn.Softmax(dim=1)(sl)
    entropy.append(-(y.exp()*y).sum().detach().numpy())  
    
d = np.argmax(np.array(entropy))
print(d, entropy[d])  

In [ ]:
# La predicción para el ejemplo más incierto:
image_test, label_test = mnist_test_data[d]
sl = model.forward(image_test.unsqueeze(0))
y = torch.nn.Softmax(dim=1)(sl)
display(y.argmax())

plt.figure(figsize=(3, 3))
plt.imshow(image_test[0, :, :].numpy(), cmap=plt.cm.Greys_r);
plt.title(label_test);

# Aumentación de datos


Clase de Lunes

# Transferencia de Aprendizaje

Ajuste fino de un modelo pre-entrenado

Clase de Lunes

# Localización y segmentación

Encontrar y segmentar objetos en imágenes

Clase de Lunes